In [527]:
#!g1.1
print(1)

1


In [519]:
#!g1.1
# Помощь в визуализации
%matplotlib inline
from IPython.display import clear_output
from tqdm.notebook import tqdm, trange
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)

# Базовые библиотеки
import os
import numpy as np
import matplotlib.pyplot as plt
import random

# Аугментации
# %pip install -qqq albumentations
import albumentations as A

# Torch
import torch
from torch import nn
from torch import optim
import segmentation_models_pytorch as smp

# Красивые графики
# %pip install wandb -qqq
import wandb

print('Ready to work')

Ready to work


# Config and settings

In [520]:
#!g1.1
from utils import Config 
cfg = Config()

cfg.data_folders_train = '/home/jupyter/mnt/datasets/tissue-patches-1024/train'
cfg.data_folders_val = '/home/jupyter/mnt/datasets/tissue-patches-1024/val'
cfg.save_folder = ''
cfg.save_name = 'UnetPP-efnb7-1024'

# Setting seed
cfg.seed = 42

# Model settings
cfg.pretrained = True
cfg.epochs = 12
cfg.lr = 1e-4

# Dataloading
cfg.batch_size = 8
cfg.val_size = 0.2

# Cross-validation
cfg.kfold = False
# cfg.n_splits = 3
# cfg.fold_number = 1

# Data loading

In [521]:
#!g1.1
cfg.pre_transforms = [
    dict(
        name="Resize",
        params=dict(
            height=512,
            width=512,
            p=1.0,
        )
    ),
]

cfg.augmentations = [
    dict(
        name="HorizontalFlip",
        params=dict(
            p=0.5)
    ),
    dict(
        name="VerticalFlip",
        params=dict(
            p=0.5)
    ),
#     dict(
#         name="ElasticTransform",
#         params=dict(
#             p=0.5)
#     ),
    dict(
        name="RandomRotate90",
        params=dict()
    ),
#     class albumentations.augmentations.geometric.transforms.ElasticTransform 
]

cfg.post_transforms = []

# Train


In [522]:
#!g1.1
cfg.device = 'cuda' if torch.cuda.is_available() else 'cpu'
cfg.model = 'UnetPlusPlus' #@param ['DeepLabV3', 'Unet', 'UnetPlusPlus']
cfg.backbone = 'efficientnet-b7' #@param [ 'None', 'resnet101', 'resnet50', 'resnet34', 'resnext101_32x4d', 'resnet152', 'efficientnet-b7', 'efficientnet-b6', 'efficientnet-b5']
cfg.in_channels = 3
cfg.num_classes = 1
cfg.layers_to_freeze = 4
cfg.max_early_stopping = 2

cfg.metric = 'DiceScore' #@param ['IoUScore', 'IoUScoreBinary', 'DiceScore']
cfg.metric_params = dict()
cfg.criterion = 'IoULoss' #@param ['IoULoss', 'BCELoss', 'DiceLoss', 'FocalLoss']
cfg.criterion_params = dict()

cfg.optimizer = 'Adam'
cfg.optimizer_params = dict(
    lr=cfg.lr
)

from utils.datagenerator import get_paths


cfg.scheduler = 'OneCycleLR'
cfg.scheduler_params = dict(
    max_lr=cfg.lr,
    epochs=cfg.epochs,
    steps_per_epoch=len(get_paths(cfg.data_folders_train)) // cfg.batch_size
)

# cfg.scheduler = None
# cfg.scheduler_params = dict(
#     lr=cfg.lr
# )

In [523]:
#!g1.1
!nvidia-smi

Sun Apr 10 11:52:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:8C:00.0 Off |                    0 |
| N/A   30C    P0    21W / 300W |      4MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!g1.1
!ls weights

In [526]:
#!g1.1
from train_functions import run

# cfg.pretrained = 'weights/UnetPP-efnb7-1024-of-IoULoss-0.18118978592387416.pth'
cfg.save(replace=False)
model = run(cfg, use_wandb=True)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /tmp/xdg_cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: werserk (use `wandb login --relogin` to force relogin)
100%|██████████| 57/57 [02:54<00:00,  3.07s/it]



Epoch #1
      Score    |    Loss
Train: 0.383507 | 0.725856
Val: 0.610855 | 0.617854

Epoch #2
      Score    |    Loss
Train: 0.694118 | 0.495158
Val: 0.749171 | 0.420326

Epoch #3
      Score    |    Loss
Train: 0.761419 | 0.367510
Val: 0.788808 | 0.349835

Epoch #4
      Score    |    Loss
Train: 0.784954 | 0.317697
Val: 0.793049 | 0.328245

Epoch #5
      Score    |    Loss
Train: 0.801975 | 0.289631
Val: 0.781562 | 0.324667

Epoch #6
      Score    |    Loss
Train: 0.813062 | 0.269914
Val: 0.789140 | 0.317154

Epoch #7
      Score    |    Loss
Train: 0.822553 | 0.251342
Val: 0.795394 | 0.310105

Epoch #8
      Score    |    Loss
Train: 0.838743 | 0.233033
Val: 0.799443 | 0.305443

Epoch #9
      Score    |    Loss
Train: 0.850412 | 0.215669
Val: 0.793946 | 0.303014

Epoch #10
      Score    |    Loss
Train: 0.857999 | 0.206147
Val: 0.804346 | 0.300623

Epoch #11
      Score    |    Loss
Train: 0.861264 | 0.200609
Val: 0.798109 | 0.305116

Epoch #12
      Score    |    Loss
Train

lr,▂▅███▇▆▄▃▂▁▁
train_loss,█▅▃▃▂▂▂▁▁▁▁▁
train_score,▁▆▇▇▇▇▇█████
val_loss,█▄▂▂▂▁▁▁▁▁▁▁
val_score,▁▆▇█▇▇██████
lr,0.0
train_loss,0.19668
train_score,0.86317
val_loss,0.30678
val_score,0.8019


In [ ]:
#!g1.1
